<a href="https://colab.research.google.com/github/Omri-Triff/Text-to-Timbre-Drum-Transfer/blob/main/models/audioLDM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div class="markdown-google-sans">

<a name="machine-learning-examples"></a>

### Featured examples

</div>

- [Retraining an Image Classifier](https://tensorflow.org/hub/tutorials/tf2_image_retraining): Build a Keras model on top of a pre-trained image classifier to distinguish flowers.
- [Text Classification](https://tensorflow.org/hub/tutorials/tf2_text_classification): Classify IMDB movie reviews as either *positive* or *negative*.
- [Style Transfer](https://tensorflow.org/hub/tutorials/tf2_arbitrary_image_stylization): Use deep learning to transfer style between images.
- [Multilingual Universal Sentence Encoder Q&A](https://tensorflow.org/hub/tutorials/retrieval_with_tf_hub_universal_encoder_qa): Use a machine learning model to answer questions from the SQuAD dataset.
- [Video Interpolation](https://tensorflow.org/hub/tutorials/tweening_conv3d): Predict what happened in a video between the first and the last frame.


In [ ]:
# 1. התקנת הספריות
print("installing dependencies...")
!pip install -q diffusers transformers accelerate scipy

# 2. בדיקה שיש לנו כרטיס מסך חזק (GPU)
import torch
if torch.cuda.is_available():
    print("✅ GPU Connected: ", torch.cuda.get_device_name(0))
else:
    print("⚠️ Warning: No GPU connected. Go to Runtime > Change runtime type > T4 GPU")

installing dependencies...
✅ GPU Connected:  Tesla T4


In [ ]:
%%writefile run_model.py
import torch
from diffusers import AudioLDMPipeline
import scipy.io.wavfile
import argparse
import os

def generate_audio(prompt, duration=5.0, steps=20, output_file="output.wav"):
    print(f"\n--- 🎵 Starting generation for: '{prompt}' ---")

    # טעינת המודל (חכם: מוריד רק אם אין ב-Cache)
    # שימוש ב-float16 לביצועים מהירים וחסכון בזיכרון
    try:
        pipe = AudioLDMPipeline.from_pretrained("cvssp/audioldm-s-full-v2", torch_dtype=torch.float16)
    except Exception as e:
        print(f"Error loading model: {e}")
        return

    # שימוש ב-GPU אם קיים
    if torch.cuda.is_available():
        pipe = pipe.to("cuda")
        print("🚀 Using NVIDIA GPU (CUDA)")
    else:
        print("🐢 Warning: Using CPU. This might be slow.")

    # היצירה עצמה
    print("Generating audio...")
    audio = pipe(prompt, num_inference_steps=steps, audio_length_in_s=duration).audios[0]

    # שמירה
    scipy.io.wavfile.write(output_file, rate=16000, data=audio)
    print(f"✅ Success! Saved to: {output_file}")

if __name__ == "__main__":
    # מאפשר לקבל פרמטרים משורת הפקודה
    parser = argparse.ArgumentParser()
    parser.add_argument("--prompt", type=str, required=True, help="Text description")
    parser.add_argument("--out", type=str, default="generated.wav", help="Output filename")
    parser.add_argument("--time", type=float, default=5.0, help="Duration in seconds")
    args = parser.parse_args()

    generate_audio(args.prompt, duration=args.time, output_file=args.out)

Writing run_model.py


In [ ]:
# שנה את הטקסט בתוך הגרשיים למה שבא לך
!python run_model.py --prompt "A dramatic drum solo in a huge hall" --out "drums.wav" --time 10


2025-12-11 10:46:33.342886: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765449993.364417    5412 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765449993.370530    5412 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765449993.385751    5412 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765449993.385776    5412 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765449993.385780    5412 computation_placer.cc:177] computation placer alr

In [ ]:
# שנה את הטקסט בתוך הגרשיים למה שבא לך
!python run_model.py --prompt "80's drums" --out "drums.wav" --time 6


2025-12-11 10:49:36.387924: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765450176.407592    6214 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765450176.413595    6214 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765450176.428625    6214 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765450176.428651    6214 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765450176.428655    6214 computation_placer.cc:177] computation placer alr

In [ ]:
from IPython.display import Audio
Audio("drums.wav")